# Evaluating Capymoa Drift Detectors

This notebook evaluates the impact of a single Concept Drift Detector on a Streaming Machine Learning model on a specified dataset.  
It measures performance in terms of accuracy, execution time, memory usage, and CPU utilization and saves them in a .csv file for benchmarking

## Install and Import Libraries

In [1]:
%%capture
import sys
if 'google.colab' in sys.modules:
    !pip install capymoa
    !pip install memory_profiler
else:
    try:
        !conda activate capymoaEnv
    except:
        print("Conda environment 'capymoaEnv' not found. Please follow setup instructions in INSTALL.md.")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import time
import psutil
import threading
from memory_profiler import memory_usage

### CAPYMOA

from capymoa.classifier import NaiveBayes, HoeffdingTree

from capymoa.datasets import Electricity, Covtype,  Hyper100k, Sensor
from capymoa.datasets import ElectricityTiny # For testing

from capymoa.drift.detectors import ADWIN, STEPD, CUSUM, PageHinkley, DDM, HDDMAverage, HDDMWeighted

from capymoa.evaluation import ClassificationEvaluator, ClassificationWindowedEvaluator
from capymoa.evaluation.results import PrequentialResults
from capymoa.evaluation.visualization import plot_windowed_results

## Evaluation

In [3]:
# @title Creating Factories
class ClassifierFactory:

    classifier_classes = {
        "NaiveBayes": NaiveBayes,
        "HoeffdingTree": HoeffdingTree
    }

    @staticmethod
    def create(classifier_type, schema):
        """Create a new classifier instance based on type."""


        if classifier_type not in ClassifierFactory.classifier_classes:
            raise ValueError(f"Unknown classifier type: {classifier_type}")

        return ClassifierFactory.classifier_classes[classifier_type](schema=schema)


class StreamFactory:

    stream_classes = {
        "Covtype": Covtype,
        "Electricity": Electricity,
        "Hyper100k": Hyper100k,
        "Sensor": Sensor,
        "ElectricityTiny": ElectricityTiny
    }

    @staticmethod
    def create(stream_type):
        """Create a new stream instance based on type."""


        if stream_type not in StreamFactory.stream_classes:
            raise ValueError(f"Unknown stream type: {stream_type}")

        # Return a new instance of the requested stream
        return StreamFactory.stream_classes[stream_type]()


class DetectorFactory:


    detector_classes = {
        "ADWIN": ADWIN,
        "STEPD": STEPD,
        "CUSUM": CUSUM,
        "PageHinkley": PageHinkley,
        "DDM": DDM,
        "HDDMAverage": HDDMAverage,
        "HDDMWeighted": HDDMWeighted
    }

    @staticmethod
    def create(detector_type):
        """Create a new detector instance based on type."""


        if detector_type == "None":
            return None

        if detector_type not in DetectorFactory.detector_classes:
            raise ValueError(f"Unknown detector type: {detector_type}")

        # Return a new instance of the requested detector
        return DetectorFactory.detector_classes[detector_type]()


In [7]:
%%capture

# @title Select Parameters

stream = "Electricity" # @param ["Covtype", "Electricity", "Hyper100k", "Sensor", "ElectricityTiny"]

classifier = "NaiveBayes" # @param ["NaiveBayes", "HoeffdingTree"]

detector = "ADWIN" # @param ["ADWIN", "STEPD", "CUSUM", "PageHinkley", "DDM", "HDDMAverage", "HDDMWeighted", "None"]


stream = StreamFactory.create(stream)
classifier = ClassifierFactory.create(classifier, stream.get_schema())
detector = DetectorFactory.create(detector)

window_size = 1.0 # 1% of dataset size


In [8]:
class Benchmarker:
    def __init__(self, stream=None, classifier=None, detector=None, window_size=1.0, cooldown_window=0, print_results=False, save_results=False, filename="results.csv"):
        if stream is None or classifier is None or detector is None:
            stream, classifier, detector, window_size, cooldown_window, print_results, save_results, filename = self.parse_args()

        self.stream = stream
        self.classifier = classifier
        self.detector = detector

        if window_size < 0 or window_size > 100:
            raise ValueError("Window size must be a value between 0 and 1.")
        self.window_size = int(stream._length * (window_size / 100)) if window_size else int(1.0 * (window_size / 100))
        if cooldown_window < 0:
            raise ValueError("Window size must be a non-negative integer.")
        self.cooldown_window = cooldown_window
        self.print_results = print_results if print_results is not None else False
        self.save_results = save_results if save_results is not None else True
        self.filename = filename if filename is not None else "results.csv"

    def evaluate_detector(self):
        i = 0
        cumulative_evaluator = ClassificationEvaluator(schema=self.stream.get_schema())
        windowed_evaluator = ClassificationWindowedEvaluator(schema=self.stream.get_schema(), window_size=self.window_size)

        changes = []
        last_detection_index = -self.cooldown_window

        while self.stream.has_more_instances():
            i += 1
            instance = self.stream.next_instance()
            y = instance.y_index
            y_pred = self.classifier.predict(instance)

            cumulative_evaluator.update(y, y_pred)
            windowed_evaluator.update(y, y_pred)

            self.classifier.train(instance)

            if self.detector is not None:
                self.detector.add_element(y)
                if self.detector.detected_change():
                    if i - last_detection_index >= self.cooldown_window:
                        self.classifier = ClassifierFactory.create(self.classifier.__class__.__name__, self.stream.get_schema())
                        last_detection_index = i
                        if self.detector.__class__.__name__ == "HDDMWeighted":
                            changes.append(i)


        # [NOTE] in capymoa==0.9.0, the add_element() method of HDDM_Weighted behaves differently to the other detectors,
        # in this evaluation function the changes are managed autonomously
        if self.detector.__class__.__name__ == "HDDMWeighted":
            self.detector.detection_index = changes

        results = PrequentialResults(
            learner=str(self.classifier),
            stream=self.stream,
            cumulative_evaluator=cumulative_evaluator,
            windowed_evaluator=windowed_evaluator
        )

        return results

    def benchmark_detector(self):
        self.stream.restart()
        cpu_samples = []

        def monitor_cpu(process, interval=0.1):
            while not stop_event.is_set():
                cpu_samples.append(process.cpu_percent(interval=None))
                time.sleep(interval)

        process = psutil.Process(os.getpid())
        stop_event = threading.Event()
        monitor_thread = threading.Thread(target=monitor_cpu, args=(process,))
        monitor_thread.start()

        start_time = time.time()
        mem_usage, results = memory_usage((self.evaluate_detector, ()), retval=True)
        end_time = time.time()

        stop_event.set()
        monitor_thread.join()

        cpu_usage = sum(cpu_samples) / len(cpu_samples) / psutil.cpu_count() if cpu_samples else 0
        execution_time = end_time - start_time
        max_mem_usage = max(mem_usage)

        results_df = pd.DataFrame([{
            "dataset": self.stream.__class__.__name__,
            "classifier": self.classifier.__class__.__name__,
            "detector": self.detector.__class__.__name__ if self.detector else "None",
            "cumulative_accuracy": results.cumulative.metrics_dict()["accuracy"],
            "cumulative_kappa": results.cumulative.metrics_dict()["kappa"],
            "windowed_accuracy": results.windowed.metrics_per_window()["accuracy"].tolist(),
            "windowed_kappa": results.windowed.metrics_per_window()["kappa"].tolist(),
            "execution_time": execution_time,
            "cpu_usage": cpu_usage,
            "memory_usage": max_mem_usage,
            "num_changes": len(self.detector.detection_index if self.detector != None else []),
        }])

        if self.save_results:
            results_df.to_csv(self.filename, mode="a", header=not pd.io.common.file_exists(self.filename), index=False)
            print(f"Results saved to {self.filename}")
        if self.print_results:
            print("Results:")
            print(results_df)

        return results_df

In [9]:
benchmark = Benchmarker(detector=detector, stream=stream, classifier=classifier,
                        window_size=window_size, cooldown_window=0,
                        print_results=False, save_results=False)


In [10]:
benchmark.benchmark_detector()

,dataset,classifier,detector,cumulative_accuracy,cumulative_kappa,windowed_accuracy,windowed_kappa,execution_time,cpu_usage,memory_usage,num_changes
0,Electricity,NaiveBayes,ADWIN,79.941296,58.666228,"[81.89845474613686, 80.3532008830022, 92.93598...","[64.07682488104908, 60.51649675353291, 84.0229...",3.254954,61.014286,895.613281,47
